In [1]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import geopandas as gpd
import sharedstreets.tile
import mercantile
from shapely.geometry import Point, shape, MultiPoint
from functools import partial
import pyproj
from shapely.ops import transform, nearest_points

#references
https://platform.sharedstreets.io/40.71297/-74.00428/17.117/
https://automating-gis-processes.github.io/2017/lessons/L7/network-analysis.html
https://automating-gis-processes.github.io/2017/lessons/L3/nearest-neighbour.html

# download the street map from OSM

In [2]:
# polygon boundry
client_ploys_gdf = gpd.read_file("GIS/TAZOfficialWCurrentForecasts.shp")

In [3]:
# project to lat-long
polygon_df_new = client_ploys_gdf.to_crs(epsg = 4326)

In [4]:
# get street network from polygon
G = ox.graph_from_polygon(polygon_df_new['geometry'].unary_union, network_type='drive')

#plot street network
#ox.plot_graph(G)

In [5]:
# save network as shapefile, two one-way links will be merged as one two-way link
# will help in runtime
#ox.save_graph_shapefile(G, filename='network-shape_MetCouncil_full')

# Read local OSM network

In [6]:
# read shapefile as geopandas df
osm_edges_gdf = gpd.read_file("data/network-shape_MetCouncil_full/edges/edges.shp")
osm_edges_gdf['mid_point'] = osm_edges_gdf['geometry'].interpolate(0.5, normalized = True)
osm_edges_gdf.shape

(230373, 20)

# Getting SharedStreets

In [7]:
# generate a GeoJSON which represents the parsed protobuf vector tiles from SharedStreets, 
# representing all tiles related to the networkx

def generate_geojson_of_coverage_area_streets(G, z=12, buffer_size = 0.015):
    geojson_master = None
    for mt in _generate_tile_coordinates(G, z, buffer_size):
        tile = sharedstreets.tile.get_tile(z, mt.x, mt.y)
        geojson = sharedstreets.tile.make_geojson(tile)

        if geojson_master is None:
            geojson_master = geojson

        else:
            # Updates both the features and references keys
            for key in ['features', 'references']:
                fs = _filter_new_objects(geojson_master, geojson, key)
                geojson_master[key].extend(fs)
    return geojson_master



def _generate_tile_coordinates(G, z, buffer_size = 0.015):
    '''
    :G: is either a networkX graph instance or a tuple of lat/long
    :z: is zoom level. Default is 12
    :buffer_size: is in degrees. Default: 0.015 is approx 1 mile, 0.03 is about 2 miles
    # parts of this are from http://kuanbutts.com/2018/06/07/sharedstreets-explore/
    # convert lat-long to XYZ tiles, use mercantile, https://media.readthedocs.org/pdf/mercantile/latest/mercantile.pdf
    '''
    mts = []
    for i, n in list(G.nodes(data=True)):
        # Now perform a buffer around the node points
        # to get a rough estimate of everything within
        # about 1 miles of the node
        p = Point(n['x'], n['y'])
        bp = p.buffer(buffer_size)  
        bpe = bp.simplify(0.005).exterior
        for x, y in zip(*bpe.coords.xy):
            mt = mercantile.tile(x, y, z)
            mts.append(mt)

    # Dedupe results
    return set(mts)


def _filter_new_objects(master, new_gj, key):
    keep = []
    gm_ids = [f['id'] for f in master[key]]
    for f2 in new_gj[key]:
        if f2['id'] not in gm_ids:
            keep.append(f2)
    return keep

In [8]:
# Generate a GeoJSON Feature Collection of the total coverage area, 1hr to run for the Met Council Core region
ssgj = generate_geojson_of_coverage_area_streets(G)

# Tag SSID to existing OSM graph

In [9]:
# creating a look up from the reference to the geometries
geometry_lookup = {}
for feature in ssgj['features']:
    i = feature['id']
    geometry_lookup[i] = feature
    
shaped_references = []
for r in ssgj['references']:
    feature = geometry_lookup[r['geometryId']]
    r['feature'] = feature
    
    # Also convert all distances to meter from centimeter
    for lr in r['locationReferences']:
        d = lr['distanceToNextRef']
        if d is not None:
            lr['distanceToNextRef'] = d/100.0

    shaped_references.append(r)

In [18]:
# Create the sharestreets edges geo dataframe
ss_edges = []
for sr in shaped_references:
    # Only do for direct edges (which should be all)
    if len(sr['locationReferences']) == 2:
        lrs = sr['locationReferences']
        if lrs[0]['sequence'] == 0:
            first = 0
            last = 1
        else:
            first = 1
            last = 0
            
        lrs = sr['locationReferences']
        fr = lrs[first]['intersectionId']
        to = lrs[last]['intersectionId']
        d = lrs[first]['distanceToNextRef']
        ss_edges.append({
            'id': sr['id'],
            'from': fr,
            'to': to,
            'length': d,
            'geometry': shape(sr['feature']['geometry'])
        })

    # This should not ever happen
    else:
        # Could actually use logger instead of a print statement
        print('Skipped an edge - not length 2')
        
ss_edges_df = pd.DataFrame(ss_edges)
ss_edges_df = ss_edges_df.drop_duplicates(subset=['id'], keep='first')
ss_edges_gdf = gpd.GeoDataFrame(ss_edges_df, geometry=ss_edges_df.geometry)

ss_edges_gdf['mid_point'] = ss_edges_gdf['geometry'].interpolate(0.5, normalized = True)
ss_edges_gdf.shape

(458030, 6)

In [22]:
# Reproject project in equal area meter
project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),  # source coordinate system
    pyproj.Proj(init='epsg:2163'))  # destination coordinate system

ss_edges_gdf_reproj = ss_edges_gdf.copy()
ss_edges_gdf_reproj['geometry'] = ss_edges_gdf_reproj['geometry'].apply(lambda g: transform(project, g))

unary_union = polygon_df_new['geometry'].unary_union
ss_edges_gdf = ss_edges_gdf[ss_edges_gdf.within(unary_union)]
ss_edges_gdf.shape

In [12]:
#ss_edges_gdf.drop('mid_point', axis = 1).to_file("data/metcouncil_ss.shp")

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [13]:
def nearest(row, geom_union, df1, df2, geom1_col, geom2_col, src_column=None):
    """Find the nearest point and return the corresponding value from specified column."""
    # Find the geometry that is closest
    nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
    # Get the corresponding value from df2 (matching is based on the geometry)
    value = df2[nearest][src_column].get_values()[0]
    return value

In [19]:
#unary_union = ss_edges_gdf['mid_point'].unary_union
unary_union = MultiPoint(ss_edges_gdf['mid_point'])
type(unary_union)

shapely.geometry.multipoint.MultiPoint

In [ ]:
nodes_to_consider = []
for i, node, from_node, to_node in zip(osm_edges_gdf['osmid'], osm_edges_gdf['mid_point'], osm_edges_gdf['from'], osm_edges_gdf['to']):
    node_p = transform(project, node)  # apply projection
    nodes_to_consider.append((i, node_p, from_node, to_node))

print('Eval {} nodes'.format(len(nodes_to_consider)))

def min_dist(point, gdf, max_d):
    # Get all possible road segments that are within 20 meters of the node
    gdf_sub = gdf[gdf.intersects(point.buffer(20))]
    gdf_sub = gdf_sub.reset_index(drop=True)
    
    # Calculate the shortest distance to all these subset segments    
    dists = []
    for geom in gdf_sub.geometry:
        d = point.distance(geom)
        dists.append(d)
        
    # Bail early if nothing to compare with
    if len(dists) == 0:
        return None
    
    # Note: min_dist is in meters
    dists = np.array(dists)
    dm = dists.min()
    
    # Do not allow "too-far" distances
    if dm > max_d:
        return None
    
    # Otherwise return the smallest distance row
    row = gdf_sub.iloc[dists == dm].head(1).squeeze()
    distance_along = row.geometry.project(point)
    return {
        'row': row,
        'distance_along': distance_along,
        'percentage_along': distance_along/row.geometry.length}

assoc_segments = []
for i, node_p, from_node, to_node in nodes_to_consider:
    closest = min_dist(node_p, ss_edges_gdf_reproj, 50)

    # Only if there is one found
    if closest is not None:
        g = closest['row'].geometry
        ss_id = closest['row']['id']
        fr_ss_id = closest['row']['from']
        to_ss_id = closest['row']['to']
        distance_along = closest['distance_along']
        percentage_along = closest['percentage_along']
    else:
        g = None
        ss_id = None
        fr_ss_id = None
        to_ss_id = None
        distance_along = None
        percentage_along = None
    assoc_segments.append({
        'osmid': i,
        'osmid_from_node' : from_node,
        'osmid_to_node' : to_node,
        'ssid_edge': ss_id,
        'ssid_from': fr_ss_id,
        'ssid_to': to_ss_id,
        'geometry': g,
        'distance_along': distance_along,
        'percentage_along': percentage_along})

assoc_segments_df = pd.DataFrame(assoc_segments)
assoc_segments_gdf_meter = gpd.GeoDataFrame(assoc_segments_df, geometry=assoc_segments_df.geometry)

Eval 230373 nodes


In [ ]:
osm_edges_gdf['ss_id'] = osm_edges_gdf.apply(nearest, 
                                        geom_union = unary_union, 
                                        df1 = osm_edges_gdf, 
                                        df2 = ss_edges_gdf, 
                                        geom1_col = 'mid_point', 
                                        geom2_col = 'mid_point',
                                        src_column='id', 
                                        axis=1)

In [ ]:
osm_edges_gdf.drop('mid_point', axis = 1).to_file("data/paired_osm_ss.shp")

In [ ]:
ss_assoc_gdf = ss_edges_gdf[ss_edges_gdf['id'].isin(osm_edges_gdf['ss_id'])]

In [ ]:
ss_assoc_gdf.plot()